In [3]:
import pandas as pd
from langchain_community.llms import Ollama
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain.agents import AgentType

In [4]:
# 1. Cargar el DataFrame
df = pd.read_csv("datos/datos_jugadores_v4/jugadores_total.csv")
print(f"✅ CSV cargado con {len(df)} filas")

✅ CSV cargado con 1908 filas


In [5]:
df.head(5)

,Unnamed: 0,player_id,player_name,team,team_id,positions,matches_played,overall,potential,value_eur,...,xg_total_per90,goals_minus_xg_per90,xg_against_per90,xg_against_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,main_position,aerial_dominance_index_per90,outdated_rating,rating
0,0,3023,Yuri Berchiche Izeta,Real Sociedad,210,['Left Back'],21,71,76,1800000.0,...,0.01,-0.02,0.0,0.0,0.46,0.0,Left Back,0.0,49.8065,65.82662180990246
1,1,3063,Danilo Luiz da Silva,Real Madrid,220,"['Right Back', 'Left Back', 'Right Wing']",24,80,86,16500000.0,...,0.04,0.04,0.0,0.0,0.25,0.0,Right Back,0.0,53.5225,68.35219356373398
2,2,3084,Christian Atsu Twasam,Málaga,223,"['Left Midfield', 'Right Midfield', 'Left Cent...",11,63,69,375000.0,...,0.41,0.24,0.0,0.0,0.00,0.0,Right Midfield,0.0,S.V,S.V
3,3,3130,Gaël Kakuta,Sevilla,213,"['Left Midfield', 'Right Wing']",2,76,78,6500000.0,...,0.00,0.00,0.0,0.0,0.96,0.0,Left Midfield,0.0,S.V,S.V
4,4,3160,Rene Krhin,Granada,1049,"['Right Defensive Midfield', 'Left Defensive M...",23,75,80,4500000.0,...,0.03,-0.04,0.0,0.0,0.18,0.0,Right Defensive Midfield,0.0,33.29875,48.27438393818753


In [6]:
# 2. Crear el modelo local de Ollama
llm = Ollama(model="llama2:chat", temperature=0)

In [7]:

prefix = """
Eres un agente inteligente especializado en análisis de datos de fútbol. 
Trabajas con un DataFrame de pandas llamado `df` que contiene información detallada de jugadores.

⚠️ **Tu única fuente de verdad es el DataFrame. No uses conocimientos externos. 
No hables de equipos, ligas o jugadores reales fuera de lo que aparece en `df`.**

Debes responder siempre siguiendo este formato:
Question: la pregunta del usuario
Thought: qué necesitas hacer
Action: python_repl_ast
Action Input: código de pandas a ejecutar
Observation: resultado del código
**Final Answer: respuesta final clara y explicativa que responda de forma directa a la pregunta del usuario**

Si no entiendes algo, intenta descomponerlo en pasos. 
Puedes usar funciones como `df[df["columna"] > valor]`, `df.sort_values(...)`, `df.groupby(...)`, etc.

Contexto de algunas métricas clave:

- "player_name": el nombre del jugador
- "rating": puntuación general del jugador
- "value_eur": valor estimado de mercado en euros
- "goals_per90": goles anotados por 90 minutos
- "progressive_passes_per90": pases progresivos por partido
- "duel_success_rate": porcentaje de duelos ganados
- "dribble_success_rate": porcentaje de regates completados
- "goal_assists_per90": asistencias por partido
- "aerial_dominance_index_per90": índice de dominio aéreo
- "goals_minus_xg_per90": diferencia entre goles y xG (positivo = sobreperformance)
- "xg_against_minus_goals_conceded_per90": indicador defensivo para porteros

Puedes usar `main_position` para filtrar por roles como "Striker", "Center Back", "Goalkeeper", etc.

Responde de forma estructurada, no inventes respuestas, y usa siempre el DataFrame real.

Ejemplo de pregunta: ¿Cuál es el delantero que tiene más goals_per90?
Respuesta: El delantero con más goals_per90 es Luis Alberto Suárez Díaz.

"""


In [8]:
prefix = """
Eres un experto en análisis de fútbol. Te doy un resumen de un DataFrame con estadísticas de jugadores y quiero que me respondas con el resultado directamente, sin explicaciones ni pasos.

DataFrame:
{tabla}

Pregunta:
¿Cuáles son los 5 jugadores con menor dribble_success_rate?

Responde directamente con los nombres.
"""

In [9]:
# 3. Crear el agente LangChain que interactúa con el DataFrame
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    handle_parsing_errors=True, 
    prefix = prefix,
    #suffix="Question: {input}\nThought:",
    #include_df_in_prompt=None  
)


In [10]:
pregunta = "¿From the dataframe, tell me the player_name of the five with the least dribble_success_rate?"
if pregunta.lower().strip() == "salir":
    print("👋 Adiós.")
try:
        respuesta = agent.invoke(pregunta)
        print("🤖", respuesta)
except Exception as e:
        print("❌ Error:", e)

❌ Error: A single string input was passed in, but this chain expects multiple inputs ({'tabla', 'input'}). When a chain expects multiple inputs, please call it by passing in a dictionary, eg `chain({'foo': 1, 'bar': 2})`


In [11]:
for column in df.columns:
    print(column)

Unnamed: 0
player_id
player_name
team
team_id
positions
matches_played
overall
potential
value_eur
wage_eur
age
dob
height_cm
weight_kg
club_jersey_number
club_loaned_from
club_contract_valid_until
nationality_id
nationality_name
preferred_foot
release_clause_eur
competition
tackle_success_rate
tackles_successful
interception_success_rate
interceptions
clearances
blocks
head_clearances
head_clearances_won
head_clearance_success_rate
headed_shots_total
headed_shots_after_duel
headed_shot_duel_rate
total_passes
completed_passes
incomplete_passes
passes_out
offside_passes
failed_passes
pass_completion_rate
avg_pass_length
ground_passes
low_passes
high_passes
ground_pass_percentage
low_pass_percentage
high_pass_percentage
crosses_total
crosses_completed
cutbacks_total
cutbacks_completed
switches_total
switches_completed
deflected_passes
goal_assists
key_passes
chances_created
through_balls_total
through_balls_completed
head_pass_percentage
right_foot_pass_percentage
left_foot_pass_percenta

In [12]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# 2. Preparar resumen reducido del DataFrame para no saturar el prompt
tabla = df.sort_values("goals_scored_per90").head(10)[["player_name", "goals_scored_per90"]].to_markdown()

# 3. Crear prompt personalizado que no siga ReAct
prompt = PromptTemplate(
    input_variables=["tabla", "pregunta"],
    template="""
Eres un experto en análisis de fútbol. Te doy un resumen de un DataFrame con estadísticas de jugadores. 
Quiero que me respondas con el resultado directamente, sin explicaciones ni pasos intermedios.

⚠️ **Tu única fuente de verdad es el DataFrame. No uses conocimientos externos. 
No hables de equipos, ligas o jugadores reales fuera de lo que aparece en `df`.**

DataFrame:
{tabla}

Pregunta:
{pregunta}

Responde solo con los nombres de los jugadores o el dato exacto.
"""
)

# 4. Cargar modelo
llm2 = Ollama(model="llama2:chat")  # o llama2, codellama...

# 5. Cadena LLM con el prompt
chain = LLMChain(llm=llm2, prompt=prompt)

# 6. Ejecutar consulta
pregunta = "¿Who is the player with the highest goals_scored_per90?"
respuesta = chain.invoke({"tabla": tabla, "pregunta": pregunta})

print("🤖", respuesta["text"])


🤖 Yuri Berchiche Izeta


In [13]:
len(tabla)

731

In [14]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["tabla", "pregunta"],
    template="""
Eres un agente experto en datos futbolísticos. Tienes acceso a un DataFrame que representa un CSV con información de jugadores. 

📌 Tu comportamiento debe cumplir las siguientes normas:
- Solo puedes responder utilizando los datos del CSV. No uses conocimiento externo.
- Cuando recibas una consulta:
  1. Detecta qué columnas están implicadas en la pregunta.
  2. Aplica los filtros necesarios sobre los datos del CSV.
  3. Devuelve solo los jugadores que cumplan los criterios.

📋 En la respuesta debes mostrar:
- Siempre: `player_name`, `team`, `value_eur`
- Además: todas las columnas **relevantes** para la consulta (por ejemplo, `dribble_success_rate` si se habla de regates).
- Presenta los datos en forma de tabla clara. Si no hay resultados, di: "No se encontraron jugadores que cumplan los filtros".

Aquí tienes una vista parcial del DataFrame:

{tabla}

Pregunta del usuario:
{pregunta}

Tu respuesta:
"""
)


In [15]:
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
import pandas as pd



# 2. Selección de columnas clave + vista parcial (para el contexto)
cols_vista = ["player_name", "team", "value_eur", "dribble_success_rate", "goals_scored_per90", "main_position"]
tabla = df[cols_vista].sort_values("dribble_success_rate").head(15).to_markdown(index=False)

# 3. Cargar modelo
llm = Ollama(model="llama2:chat")

# 4. Crear cadena con el nuevo prompt
chain = LLMChain(llm=llm, prompt=prompt)

# 5. Pregunta
pregunta = "¿Qué extremos tienen más goles por partido?"

# 6. Ejecutar
respuesta = chain.invoke({"tabla": tabla, "pregunta": pregunta})

print("🤖", respuesta["text"])


🤖 ¡Hola! Basándonos en los datos proporcionados, podemos responder a la pregunta del usuario.

Para identificar los jugadores que han marcado más goles por partido, necesitamos analizar las columnas "goals_scored_per90" y "player_name".

Entonces, los jugadores que cumplen estos criterios son:

* Darren Randolph (West Ham United) con un valor de 0.49 goles por partido.
* Mark Bunn (Aston Villa) con un valor de 0.08 goles por partido.
* Tommy Elphick (AFC Bournemouth) con un valor de 0.08 goles por partido.

A continuación, te presento los datos en forma de tabla:

| player_name | goals_scored_per90 |
| --- | --- |
| Darren Randolph | 0.49 |
| Mark Bunn | 0.08 |
| Tommy Elphick | 0.08 |

Es importante tener en cuenta que si no hay resultados, es posible que no haya jugadores que cumplan los filtros especificados. En este caso, no se encontraron jugadores que cumplan los criterios de goles por partido.


In [18]:
tabla

'| player_name                 | team             |     value_eur |   dribble_success_rate |   goals_scored_per90 | main_position     |\n|:----------------------------|:-----------------|--------------:|-----------------------:|---------------------:|:------------------|\n| Federico Barba              | VfB Stuttgart    | 250000        |                      0 |                 0.49 | Left Center Back  |\n| Shay Given                  | Stoke City       | 700000        |                      0 |                 0    | Goalkeeper        |\n| Maarten Stekelenburg        | Southampton      |      5e+06    |                      0 |                 0    | Goalkeeper        |\n| Darren Randolph             | West Ham United  |      1.8e+06  |                      0 |                 0    | Goalkeeper        |\n| Rubén Iván Martínez Andrade | Levante UD       |      3.2e+06  |                      0 |                 0    | Goalkeeper        |\n| Mark Bunn                   | Aston Villa    

In [1]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain_community.llms import Ollama

# Cargar CSV
import pandas as pd
df = pd.read_csv("datos/datos_jugadores_v5/jugadores_total.csv")

# Cargar modelo Ollama
llm = Ollama(model="codellama:7b-instruct")

# Prompt que lo fuerza a responder directamente
prefix = """
Eres un agente experto en análisis de jugadores de fútbol. Solo puedes usar el DataFrame `df` que contiene columnas como 'player_name', 'team', 'value_eur', 'goals_per90', etc.

**Solo puedes utilizar los datos que están dentro de ese DataFrame.**
**No debes usar conocimiento externo.**
No debes mencionar a jugadores que no estén en el DataFrame.
Si no puedes encontrar la respuesta en los datos, responde con: "No se encontraron resultados que cumplan los filtros."

Guía de traducción para interpretar posiciones (columna main_position):
- portero → goalkeeper
- lateral → left back, right back
- defensa central → center back
- mediocentro defensivo → defensive midfield
- mediocentro → center midfield
- mediocentro ofensivo → offensive midfield
- extremo → winger
- delantero → striker

🧠 Tu proceso siempre debe seguir este formato ReAct:

Debes seguir siempre este formato:
Question: (la pregunta del usuario)
Thought: (explica qué harás)
Action: python_repl_ast
Action Input: (código Python para pandas)
Observation: (resultado del código)
Thought: (reflexiona si ya puedes responder)
Final Answer: (responde con texto claro, usando los datos de Observation)

⚠️ No debes usar información externa, solo el DataFrame.
⚠️ Siempre termina con un Final Answer.

Cuando te pregunten, debes:
- Detectar los filtros de la consulta.
- Aplicarlos sobre `df` usando pandas.
- Mostrar como respuesta final solo una tabla con las columnas `player_name`, `team`, `value_eur` y aquellas relacionadas con la pregunta (por ejemplo `goals_per90` si se pregunta por goles).

No expliques tu razonamiento. No muestres el código ni pasos. Solo da el resultado.
"""

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prefix=prefix,
    handle_parsing_errors=True,
    allow_dangerous_code=True,
    suffix="Pregunta: {input}\nRespuesta:",
    include_df_in_prompt=None
)

# Pregunta ejemplo
pregunta = "¿Qué jugador tiene el mejor rating?"
respuesta = agent.invoke(pregunta)
print("🤖", respuesta)


`agent_scratchpad` should be a variable in prompt.input_variables. Did not find it, so adding it at the end.




> Entering new AgentExecutor chain...

Question: ¿Qué jugador tiene el mejor rating?
Thought: Debo aplicar un filtro en el DataFrame para encontrar al jugador con el mejor rating.
Action: Filtrar el DataFrame por valor de rating más alto.
Action Input: `df[df['value_eur'] == df['value_eur'].max()]`
Observation: Filtrar el DataFrame por valor de rating más alto. is not a valid tool, try one of [python_repl_ast].
Thought:
Question: ¿Qué jugador tiene el mejor rating?
Thought: Debo aplicar un filtro en el DataFrame para encontrar al jugador con el mejor rating.
Action: Filtrar el DataFrame por valor de rating más alto.
Action Input: `df[df['value_eur'] == df['value_eur'].max()]`
Observation: Filtrar el DataFrame por valor de rating más alto. is not a valid tool, try one of [python_repl_ast].
Thought:
Question: ¿Qué jugador tiene el mejor rating?
Thought: Debo aplicar un filtro en el DataFrame para encontrar al jugador con el mejor rating.
Action: Filtrar el DataFrame por valor de rating